## Задание 1.

### Обязательная часть

Будем парсить страницу со свежеми новостям на **habr.com/ru/all/.**

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 

Эти слова определяем в начале кода в переменной, например:

KEYWORDS = \['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал **не только preview-информацию статьи, но и весь текст статьи целиком**.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta

In [2]:
# ключевые слова

DESIRED_WORDS = ['визуализация', 'soft skills', 'smp', 'программист', 'интеллект']

In [3]:
# запрос на получение информации с сайта habr и извлечение данных из html страницы

result = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(result.text, 'html.parser')

In [4]:
# статьи находятся по тэгу article с атрибутом post_preview

articles = soup.find_all('article', {'class': 'post_preview'})
articles

="Вы не подписаны на этот хаб">Управление продуктом</a>, 
         </li>
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/sales/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Управление продажами'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Управление продажами</a>, 
         </li>
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/design/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Дизайн'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Дизайн</a>
 </li>
 </ul>
 <ul class="post__marks inline-list"></ul>
 <div class="post__body post__body_crop">
 <div class="post__text post__text-html post__text_v1">
       В 2014 году у нас взяли <a href="https://vc.ru/design/2927-designmodo-startup" rel="nofollow">одно из первых интервью на русском язык

In [5]:
# собираем по каждой статье заголовок, текст preview, ссылку и время публикации

news = []

for article in articles:
    title = article.find('h2', {'class': 'post__title'}).text.strip('\n')
    text_ = article.find('div', {'class': 'post__text'}).text.strip('\n')
    link = article.find('h2', {'class': 'post__title'}).find('a').get('href')
    time_ = article.find('span', {'class': 'post__time'}).get_text()[:-8]
    
    news.append({
        'время': time_,
        'заголовок': title,
        'текст': text_,
        'ссылка': link
    })
news

[{'время': 'сегодня',
  'заголовок': 'Особенности защиты беспроводных и проводных сетей. Часть 2 — Косвенные меры защиты',
  'текст': 'Продолжаем разговор о методах повышения безопасности сетей. В этой статье поговорим о дополнительных мерах безопасности и организации более защищенных беспроводных сетей.',
  'ссылка': 'https://habr.com/ru/company/zyxel/blog/516914/'},
 {'время': 'сегодня',
  'заголовок': '10 шагов к успешному выступлению на хакатоне',
  'текст': 'Константин Архипов — менеджер проектов в МТС, эксперт в инновациях, а также член жюри, автор конкурсных заданий и амбассадор «Цифрового прорыва». В прошлом году кейс, соавтором которого он являлся, занял второе место по популярности среди команд. В преддверии второго полуфинала в Приволжском IT-хабе Константин провел вебинар, в котором дал советы участникам проекта, как успешно выступить на хакатоне. Для тех, кто пропустил прямой эфир, публикуем текстовую версию выступления эксперта.\n\r\nУчастники хакатона находятся в огранич

In [6]:
# оставляем те статьи, у которых в заголовках или тексте preview есть искомые слова

news_pd = []

for new in news:
    for key, value in new.items():
        for word in DESIRED_WORDS:
            if word in new['заголовок'].lower() or word in new['текст'].lower():
                if new not in news_pd:
                    news_pd.append(new)
news_pd


[{'время': 'сегодня',
  'заголовок': 'О том, как искусственный интеллект помогает сэкономить время, или как мы встраивали ИИ в бизнес-процесс открытия счета',
  'текст': '      Всем привет! \n\r\nМы с командой представляем направление развития бизнеса с партнерами Росбанка. Сегодня хотим рассказать об успешном опыте автоматизации банковского бизнес-процесса с использованием прямых интеграций между системами, искусственного интеллекта в части распознавания образов и текста на базе GreenOCR, РФ-законодательстве и подготовке сэмплов для обучения.',
  'ссылка': 'https://habr.com/ru/company/rosbank/blog/516894/'},
 {'время': 'сегодня',
  'заголовок': '5. NGFW для малого бизнеса. Облачное управление SMP',
  'текст': ' Приветствую читателей в нашем цикле статей, который посвящен SMB Check Point, а именно модельному ряду 1500 серии. В первой части упоминалось о возможности управления вашими NGFW серии SMB с помощью облачного сервиса Security Management Portal (SMP). Наконец, пришло время расск

In [7]:
# из выбранных статей создаем датафрейм

df_news = pd.DataFrame()

for new in news_pd:
    row = {'время': new['время'], 'заголовок': new['заголовок'], 'ссылка': new['ссылка']}
    df_news = pd.concat([df_news, pd.DataFrame([row])])

df_news 

,время,заголовок,ссылка
0,сегодня,"О том, как искусственный интеллект помогает сэ...",https://habr.com/ru/company/rosbank/blog/516894/
0,сегодня,5. NGFW для малого бизнеса. Облачное управлени...,https://habr.com/ru/company/tssolution/blog/51...
0,сегодня,Левел-ап для программистов: какие навыки важно...,https://habr.com/ru/post/516898/
0,вчера,Что такое soft skills для инженера в 2020 году...,https://habr.com/ru/company/gms/blog/516882/


In [8]:
# преобразуем "время" в даты
df_news['дата'] = df_news['время'].map({'сегодня': date.today(), 
                'вчера': (date.today() + timedelta(days=-1))})

# удаляем ненужное
df_news = df_news.drop(['время'], axis=1)

# столбцы располагаем в нужном порядке
df_news = df_news[['дата', 'заголовок', 'ссылка']]
df_news

,дата,заголовок,ссылка
0,2020-08-28,"О том, как искусственный интеллект помогает сэ...",https://habr.com/ru/company/rosbank/blog/516894/
0,2020-08-28,5. NGFW для малого бизнеса. Облачное управлени...,https://habr.com/ru/company/tssolution/blog/51...
0,2020-08-28,Левел-ап для программистов: какие навыки важно...,https://habr.com/ru/post/516898/
0,2020-08-27,Что такое soft skills для инженера в 2020 году...,https://habr.com/ru/company/gms/blog/516882/


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck.

Список email-ов задаем переменной в начале кода: EMAIL = \[xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами:

<почта> - <дата утечки> - <источник утечки> - <описание утечки>.

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

Документация к API VK: https://vk.com/dev/methods, вам поможет метод wall.getGROUP = 'netology'

TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [9]:
# перечень проверяемых e-mail адресов
EMAILS = ['kirillspa@gmail.com', 'vasya@ya.ru', 'sv178@gmail.com']

# адрес, на который будет отправляться post запрос
URL = 'https://digibody.avast.com/v1/web/leaks'

df_total = pd.DataFrame()

# получаем данные по каждому e-mail из списка
for em in EMAILS:
    params = {'email': em}
    totals = requests.post(URL, json=params)

    # информацию преобразуем в json объект (словарь) и оставляем необходимое (значение ключа value)
    total = totals.json()['value']
    
    # извлекаем требуемые данные - почту, дату, источник и описание - по каждой утечке
    # собираем в датафрейм
    for tot in total:
        line = {'почта': tot['username'], 'дата': tot['leak_date'], 'источник утечки': tot['domain'], 'описание утечки': tot['leak_info']['description']}
        df_total = pd.concat([df_total, pd.DataFrame([line])])

df_total

,почта,дата,источник утечки,описание утечки
0,kirillspa@gmail.com,1513900800000,,The proliferation of stolen or leaked database...
0,kirillspa,1564012800000,,This source has been marked as sensitive due t...
0,kirillspa@gmail.com,1549411200000,,"On January 7, 2019, an online user named Sanix..."
0,kirillspa,1560384000000,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,kirka12,1477353600000,last.fm,"In March 2012, the British music streaming ser..."
0,kirillspa@gmail.com,1548720000000,,"On January 7, 2019, an online user named Sanix..."
0,kirill\_tereshenko27,1544659200000,houzz.com,"In July 2018, the housing site Houzz was alleg..."
0,kirillspa@gmail.com,1575504000000,,This source has been marked as sensitive due t...
0,kirka13,1497398400000,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
0,kirillspa,1551312000000,500px.com,"In July 2018, 500px's database was allegedly b..."


In [10]:
# преобразуем дату в "понятный" формат
df_total['дата утечки'] = df_total['дата'].map(lambda x: datetime.fromtimestamp(x/1000).isoformat())

# удаляем ненужное
df_total = df_total.drop(['дата'], axis=1)

# столбцы располагаем в нужном порядке
df_total = df_total[['почта', 'дата утечки', 'источник утечки', 'описание утечки']]
df_total

,почта,дата утечки,источник утечки,описание утечки
0,kirillspa@gmail.com,2017-12-22T03:00:00,,The proliferation of stolen or leaked database...
0,kirillspa,2019-07-25T03:00:00,,This source has been marked as sensitive due t...
0,kirillspa@gmail.com,2019-02-06T03:00:00,,"On January 7, 2019, an online user named Sanix..."
0,kirillspa,2019-06-13T03:00:00,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,kirka12,2016-10-25T03:00:00,last.fm,"In March 2012, the British music streaming ser..."
0,kirillspa@gmail.com,2019-01-29T03:00:00,,"On January 7, 2019, an online user named Sanix..."
0,kirill\_tereshenko27,2018-12-13T03:00:00,houzz.com,"In July 2018, the housing site Houzz was alleg..."
0,kirillspa@gmail.com,2019-12-05T03:00:00,,This source has been marked as sensitive due t...
0,kirka13,2017-06-14T03:00:00,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
0,kirillspa,2019-02-28T03:00:00,500px.com,"In July 2018, 500px's database was allegedly b..."


В зависимости от источника утечки данный сервис возвращает либо адрес почты, либо логин пользователя.